In [1]:
import json

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [3]:
import import_ipynb
from aux import utils
from aux import nlp
from aux import relation_extraction
from aux import defs
import preparation
import rule_base

importing Jupyter notebook from /Users/YK/mt/project/aux/utils.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/nlp.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/relation_extraction.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/defs.ipynb
importing Jupyter notebook from preparation.ipynb
importing Jupyter notebook from rule_base.ipynb


In [4]:
class RuleExplanation08(rule_base.Rule):
    name = "explanation_08"
    relation_type = "Explanation"
    reasons = {
        "COMMON_PATTERN_-_ELABORATION": 
            defs.Reason(
                1,
                "Common pattern ( -Elaboration)."
            )
    }
    
    def generate_statement(self, text, relation, verbose=False, **kwargs):
        assert(relation is not None and relation.type == "Explanation")
        info = preparation.Preprocessor.prepare_extended_info(text, relation, verbose)
        if info is None:
            utils.print_if_verbose("Extended info preparation wasn't successful.", verbose)
            return None

        if info.satellite_info.relation is None:
            return None
        
        utils.print_if_verbose(
                "Nucleus relation type: "
                f"'{utils.get_relation_type(info.nucleus_info.relation)}'.",
                verbose
            )
        utils.print_if_verbose(
            "Satellite relation type: "
            f"'{utils.get_relation_type(info.satellite_info.relation)}'.",
            verbose
        )
            
        reason = None
        
        if info.nucleus_info.relation is None:
            if info.satellite_info.relation.type == "Elaboration":
                reason = self.reasons["COMMON_PATTERN_-_ELABORATION"]

        if reason is not None:
            utils.print_if_verbose(reason.explanation, verbose)
            final_nucleus_text, final_sn_text = self._finalise_statement_parts(
                info.nucleus_preparation_result.prepared_text,
                info.satellite_preparation_result.prepared_text,
                verbose
            )
            return self._generate_statement(
                final_nucleus_text, 
                " then ", 
                final_sn_text, 
                relation, 
                info.nucleus_proximity,
                self.name,
                reason,
                verbose
            )
        else:
            return None

In [5]:
if __name__ == "__main__" and "__file__" not in globals():
    rule = RuleExplanation08()

    with open("../parsed/race/train/middle/7617.txt.tree", "rt") as f:
        tree_text = f.read()

    text, relations = relation_extraction.read_relations(
        tree_text.replace("<s>", "").replace("<P>", "")
    )

    expl = relations["Explanation"][3]
    print(text[expl.left.start:expl.right.end])

    statement =rule.generate_statement(text, expl, verbose=True)
    print("\nRESULT:")
    if statement is not None:
        print(json.dumps(statement._asdict(), indent=2))

I do n't want that kind of life , " Ma said .  After her story became known , many French student sent money to her .  Ma 's dream is the same as ever .  She hopes to go to Tsinghua University and become a reporter .  " Then , I can find poor kids like me and help them , " she said .  
Nucleus is on the left.
Nucleus's depth <= 100.
Parsing result:
(ROOT
  (S
    (S
      (NP (PRP I))
      (VP (VBP do) (RB n't)
        (VP (VB want)
          (NP
            (NP (DT that) (NN kind))
            (PP (IN of)
              (NP (NN life)))))))
    (, ,) ('' '')
    (NP (NNP Ma))
    (VP (VBD said))
    (. .)))

Constituencies:
    type  start  end  depth
0    PRP      0    1      4
1     NP      0    1      3
2    VBP      1    2      4
3     RB      2    3      4
4     VB      3    4      5
5     DT      4    5      7
6     NN      5    6      7
7     NP      4    6      6
8     IN      6    7      7
9     NN      7    8      8
10    NP      7    8      7
11    PP      6    8      6
12  